### Let's deploy an application that explores rate limiting

In [19]:
# get to the right directory
cd ~/projects/spring-cloud-gateway-workshop

In [ ]:
#check what services are setup
cf services

In [ ]:
#check the market place to see plan options are available for Spring Cloud Gateway
cf marketplace

In [ ]:
#see which apps are currently deployed
cf apps

In [16]:
#create a new service instance of the gateway
cf create-service p.gateway standard ratelimit-gateway -c '{"host": "ratelimit-gateway"}'

Creating service instance ratelimit-gateway in org scg-workshop / space demo as brian...
OK

Create in progress. Use 'cf services' or 'cf service ratelimit-gateway' to check operation status.


In [17]:
#check the create status
cf service ratelimit-gateway

Showing info of service ratelimit-gateway in org scg-workshop / space demo as brian...

name:             ratelimit-gateway
service:          p.gateway
tags:             
plan:             standard
description:      Spring Cloud Gateway for VMware Tanzu
documentation:    
dashboard:        https://ratelimit-gateway.cfapps.pcf.cloud/scg-dashboard
service broker:   scg-service-broker

Showing status of last operation from service ratelimit-gateway...

status:    create in progress
message:   create service instance started
started:   2020-08-07T03:17:54Z
updated:   2020-08-07T03:17:59Z

There are no bound apps for this service.

Upgrades are not supported by this broker.


In [20]:
#push our rate limiting app
cd ratelimiting
#cf push

In [21]:
#bind our app to the rate limiting service
cf bind-service ratelimiting ratelimit-gateway -c ./scg-config.json


Binding service ratelimit-gateway to app ratelimiting in org scg-workshop / space demo as brian...
OK

TIP: Use 'cf restage ratelimiting' to ensure your env variable changes take effect


In [22]:
#test out our no limit
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/nolimit
for i in {1..20}; do sleep .1; curl -o /dev/null -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/nolimit ; echo "\n" ; done


200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n


In [ ]:
# test out 1 per second by calling 4 times per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should get a successful response 1 in 4 times, may come across as 1 in 3 due to timing
for i in {1..20}; do sleep .25; curl -o /dev/null -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ; echo '\n';done


In [ ]:
#test out 1 per second by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should be all 200s
for i in {1..10}; do sleep 1; curl -o /dev/null -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ; echo '\n';done


In [ ]:
#test out 10 per minute by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin
for i in {1..20}; do sleep .1; curl -o /dev/null -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin ; echo "\n"; done
